# 続・detectron2 for まちカドまぞく ～カスタム・データマッパー編～

<img src="https://user-images.githubusercontent.com/33882378/79108731-e6051e80-7db1-11ea-8159-3cc55c472201.jpg">

標準以外の水増しを行う方法を試してみます。

まずは detectron2 で用意されている水増し機能を使用して見たいのですが、そのためにはトレーナーに渡す「データマッパー」というものを
作る必要があるようです。

---

### データセットの準備

前回「detectron2 for まちカドまぞく」と同じです。

In [1]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import random

In [2]:
# VoTT のエクスポートファイルや、画像が格納されているディレクトリ
BASE_DIRECTORY = './vott-json-export/'
# VoTT のエクスポートファイル名
EXPORT_FILENAME = 'Machikado-export.json'
# 訓練データに使用する割合
TRAIN_RATIO = 0.8
# 乱数シード
RANDOM_STATE = 0

In [3]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from machikado_util.Machikado_vott import get_cat_names, get_machikado_dicts

# vott エクスポートファイルの読み込み
CAT_NAME2ID, CAT_ID2NAME = get_cat_names(os.path.join(BASE_DIRECTORY, EXPORT_FILENAME))
dataset_dicts = get_machikado_dicts(os.path.join(BASE_DIRECTORY, EXPORT_FILENAME), BASE_DIRECTORY, CAT_NAME2ID)

警告: name: 59.jpg - 画像サイズが不一致であるためスキップ image_size:(268, 201), ./vott-json-export/Machikado-export.json: (600, 600)


---
### カスタムデータマッパー

> オリジナルの DatasetMapper の位置は detectron2/data/dataset_mapper.py
> 
> 以下のエラーが出る場合は、shapely をインストール
> ```
> ModuleNotFoundError: No module named 'shapely'
> ```
> ```
> pip install shapely
> ```

* T.RandomContrast なんかは、detectron2/data/transforms/transform_gen.py にありますので、一度確認してみてください。

* ポイントは、`T.Random～` という関数がジェレータの役割を担っていて、`get_transform()` で、実際の変形を加える関数を返していることです。
* コンストラクタで、ジェネレータを生成し、__call__ で、ジェネレータからトランスフォームを生成しています。
<p>トレーナーは訓練データ必要になる度にデータマッパーの def __call__(self, dataset_dict) を呼び出して、
データに変更を加えた画像や、セグメンテーション用の座標データを取得しているわけです。</p>
* 一見複雑そうですが数があるだけです。
* 形状に変形を加える場合は、順番に注意してください。イメージと座標データを同じ順番で変形しないと？な感じになります。
* あと、面倒なので今回使用するデータ以外は面倒を見ていません。
<p>（イレギュラーなものは、assert しまくっています。）</p>

In [4]:
import copy
import logging
import torch

from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils

class MachikadoDatasetMapper:
    def __init__(self, cfg, is_train=True):
        assert cfg.MODEL.MASK_ON, '今回はセグメンテーションのみを対象にする'
        assert not cfg.MODEL.KEYPOINT_ON, 'キーポイントは扱わない'
        assert not cfg.MODEL.LOAD_PROPOSALS, 'pre-computed proposals っていうのがよくわからん・・・・とりあえず無効前提で'
        
        self.cont_gen = None
        self.bright_gen = None
        self.crop_gen = None
        self.rotate_gen = None
        
        if is_train:
            if cfg.INPUT.CONTRAST.ENABLED:
                self.cont_gen = T.RandomContrast(cfg.INPUT.CONTRAST.RANGE[0], cfg.INPUT.CONTRAST.RANGE[1])
            if cfg.INPUT.BRIGHTNESS.ENABLED:
                self.bright_gen = T.RandomBrightness(cfg.INPUT.BRIGHTNESS.RANGE[0], cfg.INPUT.BRIGHTNESS.RANGE[1])
            self.extent_gen = None
            if cfg.INPUT.EXTENT.ENABLED:
                self.extent_gen = T.RandomExtent(scale_range=(1, 1), shift_range=cfg.INPUT.EXTENT.SHIFT_RANGE)
            if cfg.INPUT.CROP.ENABLED:
                self.crop_gen = T.RandomCrop(cfg.INPUT.CROP.TYPE, cfg.INPUT.CROP.SIZE)
                logging.getLogger(__name__).info('CropGen used in training: ' + str(self.crop_gen))
            if cfg.INPUT.ROTATE.ENABLED:
                self.rotate_gen = T.RandomRotation(cfg.INPUT.ROTATE.ANGLE, expand=False)
        
        self.tfm_gens = utils.build_transform_gen(cfg, is_train)
        
        self.img_format = cfg.INPUT.FORMAT
        self.mask_format = cfg.INPUT.MASK_FORMAT
        
        self.is_train = is_train

    def __call__(self, dataset_dict):
        assert 'annotations' in dataset_dict, '今回はセグメンテーションのみを対象にする'
        assert not 'sem_seg_file_name' in dataset_dict, 'パノプティックセグメンテーションは行わない'
        
        dataset_dict = copy.deepcopy(dataset_dict)
        
        image = utils.read_image(dataset_dict['file_name'], format=self.img_format)
        utils.check_image_size(dataset_dict, image)
        
        # 明るさ・コントラスト
        if self.cont_gen is not None:
            tfm = self.cont_gen.get_transform(image)
            image = tfm.apply_image(image)
        if self.bright_gen is not None:
            tfm = self.bright_gen.get_transform(image)
            image = tfm.apply_image(image)
        # アフィン
        if self.rotate_gen is not None:
            rotate_tfm = self.rotate_gen.get_transform(image)
            image = rotate_tfm.apply_image(image)
        if self.extent_gen is not None:
            extent_tfm = self.extent_gen.get_transform(image)
            image = extent_tfm.apply_image(image)
        if self.crop_gen is not None:
            crop_tfm = utils.gen_crop_transform_with_instance(
                self.crop_gen.get_crop_size(image.shape[:2]), image.shape[:2], np.random.choice(dataset_dict['annotations']))
            image = crop_tfm.apply_image(image)
        
        image, transforms = T.apply_transform_gens(self.tfm_gens, image)
        
        if self.crop_gen is not None:
            transforms = crop_tfm + transforms
        if self.extent_gen is not None:
            transforms = extent_tfm + transforms
        if self.rotate_gen is not None:
            transforms = rotate_tfm + transforms
            
        # テストの場合はアノテーションがいらないので削除して終了
        if not self.is_train:
            dataset_dict.pop('annotations', None)
            dataset_dict.pop('sem_seg_file_name', None)
            return dataset_dict

        image_shape = image.shape[:2]  # h, w
        dataset_dict['image'] = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)))

        annos = [utils.transform_instance_annotations(obj, transforms, image_shape, keypoint_hflip_indices=None)
                 for obj in dataset_dict.pop('annotations')
                 if obj.get("iscrowd", 0) == 0]

        instances = utils.annotations_to_instances(annos, image_shape, mask_format=self.mask_format)

        # マスクからバウンディングボックスを作成
        if self.crop_gen and instances.has("gt_masks"):
            instances.gt_boxes = instances.gt_masks.get_bounding_boxes()

        dataset_dict["instances"] = utils.filter_empty_instances(instances)

        return dataset_dict

### カスタマイズした設定

カスタムした分増えた設定をどうしようかと考えたが、cfg にくっつければ良いのでは？と思ったので追加する関数を作った。

In [5]:
from detectron2.config import CfgNode as CN

# カスタムした分の設定を追加する
def append_custom_cfg(cfg):
    cfg.INPUT.ROTATE = CN()
    cfg.INPUT.CONTRAST = CN()
    cfg.INPUT.BRIGHTNESS = CN()
    cfg.INPUT.EXTENT = CN()

    cfg.INPUT.ROTATE.ENABLED = True
    cfg.INPUT.ROTATE.ANGLE = [-20, 20]
    cfg.INPUT.CONTRAST.ENABLED = True
    cfg.INPUT.CONTRAST.RANGE = (0.5, 1.5)
    cfg.INPUT.BRIGHTNESS.ENABLED = True
    cfg.INPUT.BRIGHTNESS.RANGE = (0.8, 1.2)
    cfg.INPUT.EXTENT.ENABLED = True
    cfg.INPUT.EXTENT.SHIFT_RANGE = (0.2, 0.2)

* 画像の入力サイズを変更しています。
* これによって精度が向上しました。（mAP 70% -> 80%）
<p>（アニメ画像なのでそもそも標準の解像度はいらないんですよね・・・）</p>
* また、画像が小さくなったので、バッチ処理枚数が増えます。（バッチ枚数が精度向上に貢献している？）
<p>（バッチ数については、うちの GPU じゃこれ以上検証無理かな？）</p>
* ちなみに、MIN、MAX の意味は、 detectron2/data/transforms/transform_gen.py の ResizeShortestEdge を参照してください。

In [6]:
from detectron2.config import get_cfg

cfg = get_cfg()
append_custom_cfg(cfg)

cfg.MODEL.MASK_ON = True

# インプットサイズを変更しています！
cfg.INPUT.MIN_SIZE_TRAIN = 400
cfg.INPUT.MAX_SIZE_TRAIN = 640
cfg.INPUT.MIN_SIZE_TEST = 400
cfg.INPUT.MAX_SIZE_TEST = 640

cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.SIZE = [0.8, 0.8]

---
### テスト表示

画像サイズを縮小していますが十分でしょう。

In [ ]:
from machikado_util.plot_dataset_mapper import plot_dataset_mapper

# random.seed(RANDOM_STATE)
mapper = MachikadoDatasetMapper(cfg, is_train=True)
plot_dataset_mapper(dataset_dicts, mapper)